In [31]:
import gym
import torch
from torch import nn, optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy
import random
from statistics import mean

In [2]:
print(torch.cuda.is_available())

True


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [90]:
class Network(nn.Module):
    def __init__(self,seed):
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(8, 256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,64)
        self.fc4 = nn.Linear(64, 4)
    
    def forward(self, state):
        h = F.relu(self.fc1(state))
        h = F.relu(self.fc2(h))
        h = F.relu(self.fc3(h))
        y = self.fc4(h)
        return y

In [114]:
class QLearningAgent(object):
    def __init__(self,alpha,gamma,epsilon,n_eps,N,C,M,seed):
        self.memory = []
        self.memory_max = N
        self.target_update = C
        self.Q_t = Network(seed).to(device)
        self.Q = Network(seed).to(device)
        self.alpha = alpha
        self.optimizer = optim.SGD(self.Q.parameters(), lr=self.alpha)
        self.gamma = gamma
        self.epsilon = epsilon
        self.seed = seed
        self.C = C
        self.n_eps = n_eps
        self.mini_batch_size = M
        self.env = gym.make('LunarLander-v2')
        self.env.seed(seed)
    
    def store_memory(self,state,action,reward,next_state,done):
        reward = np.array([reward],dtype = float)
        action = np.array([action],dtype = int)
        done = np.array([done],dtype = int)
        self.memory.append((state,action,reward,next_state,done))
    
    def sample_memory(self,M):
        batch = np.array(random.sample(self.memory, k=M),dtype = object)
        batch = batch.T
        batch = batch.tolist()
#         print(batch[1])
        return (torch.tensor(batch[0]).to(device),torch.tensor(batch[1],dtype=torch.int64).to(device),torch.tensor(batch[2],dtype = torch.float).to(device),torch.tensor(batch[3]).to(device),torch.tensor(batch[4]).to(device))
    
    def solve(self):
        states = self.env.observation_space
        actions = self.env.action_space
        np.random.seed(self.seed)
        count = 0
        scores = []
        for eps in range(self.n_eps):
            state = self.env.reset()
            score = 0
            for i in range(1000000):
                greed = np.random.random()
                #Feed Forward once to predict the best action for current state
                self.Q.eval()
                with torch.no_grad():
                    weights = self.Q(torch.tensor(state).to(device))
#                     print(weights)
                self.Q.train()
                if greed < self.epsilon:
                    action = np.random.randint(0, 4)
                else:
                    action = np.argmax(weights.detach().cpu().numpy())
                next_state, reward, done, data = self.env.step(action)
                score+=reward
                self.store_memory(state,action,reward,next_state,done)
                
                if len(self.memory)<self.mini_batch_size:
                    break
                else:
                    transitions = self.sample_memory(self.mini_batch_size)
                
                states,actions,rewards,next_states,dones = transitions
                Q_t = self.Q_t(next_states).detach()
                Q_tmax = Q_t.max(1)[0].unsqueeze(1)
                case2 = rewards + self.gamma * Q_tmax
                case1 = rewards
#                 y_j = reward + (gamma * Q_tmax * (1-done))  
                y = torch.where(dones<1,case2,case1)    
                Q = self.Q(states).gather(1, actions)
                print(y)
                print(Q)
                loss = F.mse_loss(Q, y)
#                 print(loss)
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                count+=1
                if count == self.C:
                    count = 0
#                     self.Q_t = deepcopy(self.Q)
                    self.Q_t.load_state_dict(self.Q.state_dict())
                state = deepcopy(next_state)
                if done:
                    break
            scores.append(score)
        print(mean(scores))


In [115]:
agent = QLearningAgent(0.001,0.99,0.7,300,300,50,64,5)

In [116]:
agent.solve()

tensor([[-2.8034],
        [ 0.1124],
        [-2.0223],
        [ 0.8182],
        [-1.5688],
        [-0.7718],
        [ 0.6705],
        [ 0.0831],
        [-2.0394],
        [-1.8530],
        [ 1.4764],
        [-0.6657],
        [-0.4819],
        [ 2.2737],
        [-2.4323],
        [-1.9792],
        [ 3.3549],
        [-1.6846],
        [-1.9973],
        [ 1.9711],
        [-1.5296],
        [-1.5450],
        [-1.9230],
        [-2.0575],
        [ 0.6077],
        [-1.7831],
        [-0.6417],
        [ 1.5677],
        [-1.1218],
        [ 0.2709],
        [-1.9832],
        [-0.7079],
        [ 0.5161],
        [ 1.7517],
        [ 0.2448],
        [ 0.6231],
        [-0.7676],
        [ 1.7187],
        [ 0.4347],
        [-1.0096],
        [-0.8431],
        [-1.7115],
        [ 1.5441],
        [ 0.4616],
        [-1.8827],
        [ 0.3167],
        [-1.5262],
        [-0.6446],
        [-1.5263],
        [-1.1352],
        [-1.0047],
        [-0.8245],
        [ 1.

tensor([[ 0.0451],
        [ 0.0192],
        [ 0.0434],
        [ 0.0195],
        [ 0.0429],
        [-0.1406],
        [ 0.0498],
        [ 0.0487],
        [ 0.0407],
        [ 0.0300],
        [ 0.0406],
        [ 0.0139],
        [-0.0054],
        [ 0.0146],
        [ 0.0515],
        [ 0.0491],
        [ 0.0145],
        [-0.0024],
        [-0.1541],
        [ 0.0176],
        [ 0.0332],
        [-0.0112],
        [-0.0055],
        [-0.0057],
        [ 0.0360],
        [-0.1610],
        [-0.1412],
        [ 0.0459],
        [ 0.0435],
        [-0.0040],
        [ 0.0315],
        [-0.0062],
        [ 0.0229],
        [ 0.0258],
        [ 0.0220],
        [ 0.0444],
        [-0.0041],
        [ 0.0475],
        [ 0.0484],
        [ 0.0304],
        [ 0.0522],
        [ 0.0220],
        [ 0.0393],
        [ 0.0096],
        [-0.0032],
        [ 0.0435],
        [-0.1297],
        [ 0.0440],
        [ 0.0505],
        [-0.0067],
        [ 0.0500],
        [ 0.0495],
        [ 0.

        [-1.0047]], device='cuda:0')
tensor([[ 0.0427],
        [ 0.0491],
        [ 0.0463],
        [ 0.0494],
        [-0.1317],
        [ 0.0492],
        [ 0.0249],
        [-0.0147],
        [ 0.0394],
        [-0.0138],
        [-0.1427],
        [ 0.0255],
        [ 0.0339],
        [ 0.0328],
        [ 0.0436],
        [-0.1362],
        [-0.0100],
        [ 0.0296],
        [ 0.0521],
        [ 0.0431],
        [-0.0159],
        [ 0.0417],
        [-0.0106],
        [ 0.0192],
        [-0.1172],
        [-0.0093],
        [-0.0140],
        [-0.0139],
        [-0.0183],
        [ 0.0342],
        [ 0.0182],
        [ 0.0124],
        [ 0.0523],
        [ 0.0483],
        [-0.0085],
        [ 0.0108],
        [-0.0103],
        [-0.0110],
        [-0.0133],
        [ 0.0382],
        [ 0.0188],
        [-0.0094],
        [-0.0115],
        [-0.1304],
        [ 0.0500],
        [ 0.0247],
        [-0.0140],
        [ 0.0148],
        [-0.0136],
        [ 0.0450],
        [ 0.0

tensor([[-1.1352],
        [-1.5688],
        [-1.7831],
        [ 0.5537],
        [-0.8431],
        [-0.8245],
        [ 0.6870],
        [ 1.9711],
        [-0.5570],
        [-2.4323],
        [ 0.0831],
        [-0.5670],
        [ 0.4616],
        [-1.5262],
        [ 0.6705],
        [-1.1821],
        [-1.8827],
        [ 0.4022],
        [-0.7718],
        [ 1.3948],
        [ 1.1600],
        [-1.7584],
        [ 3.3549],
        [-1.1218],
        [ 0.2069],
        [-0.7676],
        [ 1.3502],
        [ 1.4764],
        [-1.9295],
        [-2.6403],
        [-3.5008],
        [-0.5040],
        [ 1.0774],
        [ 0.0957],
        [ 1.7517],
        [-1.8409],
        [-1.0096],
        [-2.3148],
        [ 0.6231],
        [ 1.7187],
        [-2.0965],
        [-1.8530],
        [-1.9230],
        [-1.9832],
        [ 0.2448],
        [-2.0223],
        [ 2.2737],
        [-0.6657],
        [ 0.0563],
        [ 1.1011],
        [-0.6446],
        [-1.6846],
        [ 0.

tensor([[-0.8431],
        [-2.6403],
        [-0.3783],
        [-1.5688],
        [-1.5263],
        [-0.5040],
        [ 0.5537],
        [-3.5008],
        [ 0.6231],
        [-1.7584],
        [-2.3148],
        [ 0.2138],
        [-0.7718],
        [-2.8999],
        [ 0.3167],
        [-0.6446],
        [ 3.3549],
        [ 0.0563],
        [ 0.0831],
        [-1.8409],
        [ 1.6637],
        [-0.2519],
        [ 1.5441],
        [-1.7920],
        [ 0.8182],
        [ 1.1600],
        [ 0.4347],
        [-2.0575],
        [ 2.2973],
        [ 0.0957],
        [-2.8034],
        [-0.4819],
        [-1.7831],
        [ 2.2737],
        [ 0.3849],
        [-1.3660],
        [-2.0965],
        [ 1.0774],
        [-1.0096],
        [ 0.2709],
        [-1.1476],
        [ 1.2753],
        [ 0.1384],
        [-0.6417],
        [-1.6846],
        [ 0.5773],
        [-0.9090],
        [-1.7115],
        [-0.5670],
        [-0.7676],
        [ 1.4764],
        [-1.6344],
        [ 0.

...

tensor([[-0.1966],
        [-0.1910],
        [-0.2087],
        [-0.1118],
        [-0.1580],
        [-0.1869],
        [-0.1897],
        [-0.2667],
        [-0.1943],
        [-0.1883],
        [-0.0963],
        [-0.2220],
        [-0.1225],
        [-0.1895],
        [-0.1460],
        [-0.0992],
        [-0.0972],
        [-0.2346],
        [-0.1941],
        [-0.1945],
        [-0.1333],
        [-0.2768],
        [-0.0998],
        [-0.0949],
        [-0.2510],
        [-0.0931],
        [-0.1448],
        [-0.1331],
        [-0.1948],
        [-0.1816],
        [-0.2061],
        [-0.0980],
        [-0.0960],
        [-0.0996],
        [-0.0916],
        [-0.2967],
        [-0.1410],
        [-0.1353],
        [-0.1308],
        [-0.2945],
        [-0.1413],
        [-0.1439],
        [-0.1819],
        [-0.2788],
        [-0.1482],
        [-0.0937],
        [-0.1950],
        [-0.1892],
        [-0.1223],
        [-0.0939],
        [-0.1355],
        [-0.2951],
        [-0.

tensor([[-0.3095],
        [-0.7672],
        [-0.3082],
        [-0.1685],
        [-0.2689],
        [-0.1011],
        [-0.0982],
        [-0.0982],
        [-0.1002],
        [-0.3017],
        [-0.1003],
        [-0.1057],
        [-0.2487],
        [-0.1609],
        [-0.1206],
        [-0.1006],
        [-0.0992],
        [-0.1673],
        [-0.2289],
        [-0.1017],
        [-0.3085],
        [-0.2322],
        [-0.1046],
        [-0.2914],
        [-0.1659],
        [-0.1489],
        [-0.1402],
        [-0.1505],
        [-0.3196],
        [-0.2343],
        [-0.1627],
        [-0.1048],
        [-0.3097],
        [-0.2856],
        [-0.2340],
        [-0.3060],
        [-0.1543],
        [-0.2947],
        [-0.2236],
        [-0.1600],
        [-0.1504],
        [-0.1010],
        [-0.3112],
        [-0.2510],
        [-0.2758],
        [-0.3045],
        [-0.2325],
        [-0.0995],
        [-0.1009],
        [-0.3113],
        [-0.2188],
        [-0.3071],
        [-0.

tensor([[ 1.1469e+00],
        [-2.3412e-01],
        [-1.7763e+00],
        [-2.7101e+00],
        [ 4.6354e+01],
        [ 6.8951e-01],
        [ 7.0605e-01],
        [ 8.6278e-02],
        [ 5.2303e+00],
        [ 2.2026e+00],
        [-7.1515e-01],
        [-8.3509e-01],
        [-2.0429e+00],
        [ 1.7242e+00],
        [ 7.1539e-03],
        [ 2.8274e-01],
        [-3.6675e+00],
        [-6.2148e-01],
        [-1.2884e+00],
        [-1.2162e+00],
        [-1.4797e+00],
        [-3.6623e+00],
        [-2.6987e+00],
        [-2.4696e+01],
        [ 8.7146e-01],
        [-7.0513e+00],
        [-2.9111e+00],
        [-1.9111e+00],
        [ 7.0533e-01],
        [-2.2842e+00],
        [-2.2892e+00],
        [-4.0282e-01],
        [-3.3211e-01],
        [-1.2101e+00],
        [-3.4509e+00],
        [-1.4860e+00],
        [-1.5807e+00],
        [-2.0720e+00],
        [-4.0370e+00],
        [-1.4208e+00],
        [-5.8192e-01],
        [-2.4558e+00],
        [-7.4069e-01],
        [-3

tensor([[-0.0963],
        [-0.2090],
        [-0.2067],
        [-0.3347],
        [-0.1851],
        [-0.3226],
        [-0.3148],
        [-0.3490],
        [-0.1978],
        [-0.2936],
        [-0.0990],
        [-0.3141],
        [-0.2047],
        [-0.2098],
        [-0.1965],
        [-0.3432],
        [-0.3018],
        [-0.2979],
        [-0.0988],
        [-0.2043],
        [-0.3308],
        [-0.3068],
        [-0.4813],
        [-0.2089],
        [-0.2086],
        [-0.0997],
        [-0.3396],
        [-0.0947],
        [-0.4319],
        [-0.3072],
        [-0.2098],
        [-0.2836],
        [-0.3420],
        [-0.0994],
        [-0.3031],
        [-0.0961],
        [-0.1861],
        [-0.3232],
        [-0.1147],
        [-0.2024],
        [-0.3092],
        [-0.3088],
        [-0.2096],
        [-0.1977],
        [-0.1020],
        [-0.0901],
        [-0.2062],
        [-0.1010],
        [-0.2862],
        [-0.3416],
        [-0.3068],
        [-0.1992],
        [-0.

KeyboardInterrupt: 

[ 0.0491, -0.1246,  0.0294, -0.0133]